## Premier partie: matrices

In [1]:
class Matrice:
    def __init__(self, coefficients):
        if len(coefficients) == 0:
            raise RuntimeError("Matrice vide")
        self.nblignes = len(coefficients)
        self.nbcolonnes = len(coefficients[0])
        for l in coefficients:
            if len(l) != self.nbcolonnes:
                raise RuntimeError("Lignes de longueur différente")
        self.coeffs = coefficients
        
    def __repr__(self):
        larg = max(len(str(c)) for ligne in self.coeffs for c in ligne)
        aff = ''
        for i, l in enumerate(self.coeffs):
            if self.nblignes == 1:
                delims = '〈%s 〉'
            elif i == 0:
                delims = '/%s \\\n'
            elif i == self.nblignes - 1:
                delims = '\\%s /'
            else:
                delims = '|%s |\n'
            aff += delims % ''.join(' '*(larg-len(str(c))+1) + str(c)
                                    for c in l)
        return aff
    
    def __str__(self):
        larg = max(len(str(c)) for ligne in self.coeffs for c in ligne)
        templ = '{:>%d}' % (larg + 1)
        aff = ''
        for i, l in enumerate(self.coeffs):
            aff += '|' + ''.join(templ.format(c) for c in l) + ' |\n'
        return aff
    
    def __eq__(self, other):
        '''
        Teste l'égalité de deux matrices coefficient par coefficient.
        Appelé implicitement par A == B.
        '''
        return (self.nblignes == other.nblignes
                and self.nbcolonnes == other.nbcolonnes
                and all(self.coeffs[i][j] == other.coeffs[i][j]
                   for i in range(self.nblignes)
                   for j in range(self.nbcolonnes)))

In [2]:
Matrice([[1,0],[0,10]])

/  1  0 \
\  0 10 /

In [3]:
Matrice([[123,213,-10], [23, 39, 0], [1, -1, 3]])

/ 123 213 -10 \
|  23  39   0 |
\   1  -1   3 /

In [4]:
m = Matrice([[1,2]])

In [5]:
m

〈 1 2 〉

In [6]:
str(m)

'| 1 2 |\n'

In [7]:
import random

def rand_mat(lignes, colonnes, coeffs, min=1, max=1, symetrique=False):
    if symetrique:
        assert lignes == colonnes, "Une matrice symétrique doit être carrée"
        longueur = lignes * (lignes + 1) // 2
    else:
        longueur = lignes * colonnes
    # on initialise un tableau avec coeffs valeurs aléatoires entre min et max
    # et (longueur - coeffs) zéros
    coefficients = [random.randint(min, max) for i in range(coeffs)] + [0]*(longueur - coeffs)
    # On permute le tableau.
    # On utilise ici le "Knuth shuffle" pour générer une permutation aléatoire uniforme.
    # Exercice: prouver que le Knuth shuffle génère une permutation aléatoire uniforme.
    for i in range(longueur):
        j = random.randrange(i, longueur)
        coefficients[i], coefficients[j] = coefficients[j], coefficients[i]
    # On met les coefficients dans un tableau lignes × colonnes
    mat = [[0]*colonnes for i in range(lignes)]
    for i in range(lignes):
        if symetrique:
            start = i
        else:
            start = 0
        for j in range(start, colonnes):
            mat[i][j] = coefficients.pop()
    if symetrique:
        for i in range(lignes):
            for j in range(0,i):
                mat[i][j] = mat[j][i]
        
    # On met les coefficients dans une matrice
    return Matrice(mat)

In [8]:
rand_mat(2,3,3)

/ 0 1 0 \
\ 1 1 0 /

In [9]:
rand_mat(10,10,30,-10,10)

/   0  -3   0   0  -1   0   0   0   0   5 \
|   0   0   0   0   0   0   0   9   0  -4 |
|  -7   2   0   0   0   0   0  -3   0   0 |
|   0   0   0   0   0   0   1   8   0  -7 |
|   0   0   0   0   0   0   0   0   0   0 |
|   5   0   0   0   0  -3   8   0   0   0 |
|   0   0   3   7   0   0   1   0   0   0 |
|  -8   0   0   5   0   2   6   0   0   0 |
|   0   0 -10   0   0   0   0   0   0   1 |
\ -10   0   0   0   0   0   0   6   9   0 /

In [10]:
rand_mat(10,10,30,-10,10, symetrique=True)

/   0   2   0   8  -5  -7   0   0  -2  -1 \
|   2   5   0   0   4  -8 -10   0   0   3 |
|   0   0  -1   0   0   0   0   9   0  -2 |
|   8   0   0   0   6  -2   0   0  -6   4 |
|  -5   4   0   6  -7   0   7   0  -5   0 |
|  -7  -8   0  -2   0  -4   0  -1   2  -1 |
|   0 -10   0   0   7   0   6   0  10   0 |
|   0   0   9   0   0  -1   0   0   2  -9 |
|  -2   0   0  -6  -5   2  10   2   0   0 |
\  -1   3  -2   4   0  -1   0  -9   0   0 /

## Deuxième partie: graphes

In [11]:
class Noeud:
    def __init__(self, nom):
        self.nom = nom
        self.voisins = []
        
    def __repr__(self):
        return "(%s)→[%s]" % (self.nom, 
                              ",".join(v.nom 
                                       for v in self.voisins))

class Graphe:
    def __init__(self, noeuds):
        self.noeuds = noeuds
        
    def __repr__(self):
        return "\n".join(repr(n) for n in self.noeuds)
    
    def adjacence(self):
        l = len(self.noeuds)
        adj = [[0]*l for _ in range(l)]
        for i, n in enumerate(self.noeuds):
            for m in n.voisins:
                adj[i][self.noeuds.index(m)] = 1
        return Matrice(adj)

In [12]:
a = Noeud("A")
b = Noeud("B")
c = Noeud("C")
d = Noeud("D")
a.voisins = [b, d]
b.voisins = [a, c]
c.voisins = [a, b]
d.voisins = [b]
G = Graphe([a,b,c,d])
G

(A)→[B,D]
(B)→[A,C]
(C)→[A,B]
(D)→[B]

In [13]:
def graphe(mat):
    assert mat.nblignes == mat.nbcolonnes, "La matrice n'est pas carrée."
    assert all(c in (0,1) 
               for l in mat.coeffs for c in l), "La matrice n'est pas binaire"
    noeuds = [Noeud(str(i+1)) 
              for i in range(mat.nblignes)]
    for i, l in enumerate(mat.coeffs):
        for j, c in enumerate(l):
            if c:
                noeuds[i].voisins.append(noeuds[j])
    return Graphe(noeuds)

In [14]:
m = rand_mat(5,5,10)
m

/ 0 0 1 1 0 \
| 1 0 1 0 0 |
| 1 0 0 0 0 |
| 1 0 1 0 0 |
\ 1 1 1 0 0 /

In [15]:
G = graphe(m)
G

(1)→[3,4]
(2)→[1,3]
(3)→[1]
(4)→[1,3]
(5)→[1,2,3]

In [16]:
m2 = G.adjacence()
m2

/ 0 0 1 1 0 \
| 1 0 1 0 0 |
| 1 0 0 0 0 |
| 1 0 1 0 0 |
\ 1 1 1 0 0 /

In [17]:
m2 == m

True

## Troisième partie: parcours de graphes

In [18]:
# Pour simplifier le calcul des composantes connexes,
# on supposera que G est non-orienté.
# On laisse en exercice l'adaptation au cas orienté.
def parcours(G):
    # On marque tous les sommets
    for v in G.noeuds:
        v._dist = None     # distance de l'origine
        v._comp = None     # composante connexe
                           # (chaque composante sera marquée par un entier)
        
    G._comps = 0          # compteur des composantes
    for v in G.noeuds:
        if v._comp is None:
            v._comp = G._comps
            G._comps += 1
            v._dist = 0
            pile = [v]
            while pile:         # pile équivaut à True ssi elle est non-vide
                a = pile.pop(0)
                for b in a.voisins:
                    if b._dist is None:
                        b._dist = a._dist + 1
                        b._comp = a._comp
                        pile.append(b)

In [56]:
G = graphe(rand_mat(15,15,20,symetrique=True))
parcours(G)

In [57]:
G._comps

2

In [58]:
for i in range(G._comps):
    print("Composante #%d:" % i)
    for n in G.noeuds:
        if n._comp == i:
            print(n)
    print()

Composante #0:
(1)→[5,7,9,11]
(2)→[9,12]
(3)→[5,7,9]
(5)→[1,3]
(6)→[9,13]
(7)→[1,3,7,12,15]
(9)→[1,2,3,6,11,13]
(11)→[1,9]
(12)→[2,7]
(13)→[6,9]
(15)→[7]

Composante #1:
(4)→[8,10]
(8)→[4,10]
(10)→[4,8,14]
(14)→[10]

